In [3]:
!pip install datasets openai tiktoken tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.9 MB/s eta 0:00:0000:0100:01m


In [5]:
import os
import openai

In [7]:
messages = [{"role": "user", "content":"日本で一番高い山は？"}]
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages= messages,
    temperature=0.0
)

print(completion['choices'][0]["message"]["content"])

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
